# Prerequisites

## For Server 
- git clone https://github.com/most-medicalrecord -b develop
- cd most-medicalrecord
- make develop
- make sync
- make run

##For testing API with this notebook## 
- **Requests** library <http://docs.python-requests.org/en/latest/>

## For testing pyEHR API endpoint
- Download ova VirtualMachine from <http://156.148.18.70/files/pyEHR.ova>
- Run VirtualMachine (virtualbox or vmware importing vmdk virtual disk in new virtual machine)
- Configure VirtualMachine networking with NAT or BRIDGE
- Wait for services start (up to 5 minutes)
- Test dbservice status calling <http://virtualmachine:8080/check/status/dbservice>
- Configure PYEHR_DB_SERVICE_IP on django settings.py
- stop SERVER and restart with **make run**

In [1]:
#Get Oauth2 Access Token specifying existing taskgroup_uuid

import requests
import json
import pprint

server_port = 8000

arguments = {
             'client_id': '8c96bf8cea26fa555fa8',
             'client_secret': '4fd1f508b7b03fba6509da4c193157d7a2b20838',
             'grant_type': 'password',
             'username': 'admin',
             'password': 'admin',
             'taskgroup': '5dw2x3jfkftxue5a5izw6yiplbbn4dlo'
             }
r = requests.post(
                  'http://127.0.0.1:{server_port}/oauth2/access_token/'.format(server_port=server_port), 
                  data=arguments
                  )
print r.text

access_token = json.loads(r.text)['access_token']

{"access_token": "6772f8e4a167eb35d875e9b7552603cca7b87fd3", "token_type": "Bearer", "expires_in": 31535999, "refresh_token": "87060aec592942e8e921d576e1c1c0ff2e985809", "scope": "read"}


In [2]:
# Get EHR DBService status

payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/status/'.format(server_port=server_port), 
    params=payload)
print r.text

DBService daemon running


In [3]:
# Create patient record
# The patient uuid is the first of the most-demo database (CARLO VERDI)
payload = {'access_token': access_token, 'demographic_uuid': 'afbab3d293b643833d35cb972f26642cf77f1db4'}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text
data = json.loads(r.text)
patient_id = data['patient']['uuid']
print 'patient id: %s' % patient_id

{"patient": {"demographic_uuid": "afbab3d293b643833d35cb972f26642cf77f1db4", "ehr_uuid": "plqa4mckhd33nslx4vplm5hklwe36fqi", "uuid": "2tppdtslwxwa77j6ozqdgidtvdgcmmwl"}, "success": true}
patient id: 2tppdtslwxwa77j6ozqdgidtvdgcmmwl


In [4]:
#Create ehr patient record 
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
    params=payload
)
print r.text


{"RECORD": {"record_id": "plqa4mckhd33nslx4vplm5hklwe36fqi", "active": true, "ehr_records": [], "creation_time": 1493717432.802065, "last_update": 1493717432.802065}, "SUCCESS": true}


In [5]:
#Create ehr medical record
params = {'access_token': access_token}

ehr_data = {
  "archetype_class": "openEHR-EHR-SECTION.report_demo_most.v1",
  "archetype_details": {
    "items": {
      "at0001": {
        "archetype_class": "openEHR-EHR-OBSERVATION.fast_scan.v1",
        "archetype_details": {
          "data": {
            "at0001": [
              {
                "events": [
                  {
                    "at0002": {
                      "data": {
                        "at0003": [
                          {
                            "items": {
                              "at0004": {
                                "items": {
                                  "at0005": {
                                    "value": {
                                      "text": "Bla Bla Bla"
                                    }
                                  },
                                  "at0006": {
                                    "value": {
                                      "selected": True
                                    }
                                  }
                                }
                              },
                              "at0007": {
                                "items": {
                                  "at0008": {
                                    "value": {
                                      "text": "Yada yada"
                                    }
                                  },
                                  "at0009": {
                                    "value": {
                                      "selected": False
                                    }
                                  }
                                }
                              },
                              "at0010": {
                                "items": {
                                  "at0011": {
                                    "value": {
                                      "text": "Something, somtehing"
                                    }
                                  },
                                  "at0012": {
                                    "value": {
                                      "selected": True
                                    }
                                  }
                                }
                              },
                              "at0013": {
                                "items": {
                                  "at0014": {
                                    "value": {
                                      "text": "Ill Illr"
                                    }
                                  },
                                  "at0015": {
                                    "value": {
                                      "selected": False
                                    }
                                  }
                                }
                              }
                            }
                          }
                        ]
                      }
                    }
                  }
                ]
              }
            ]
          }
        }
      }
    }
  }
}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/'.
        format(server_port=server_port, patient_id=patient_id), 
    params=params, json=ehr_data
)
print r.text

{"RECORD": {"ehr_data": {"archetype_details": {"items": {"at0001": {"archetype_details": {"data": {"at0001": [{"events": [{"at0002": {"data": {"at0003": [{"items": {"at0013": {"items": {"at0014": {"value": {"text": "Ill Illr"}}, "at0015": {"value": {"selected": false}}}}, "at0010": {"items": {"at0012": {"value": {"selected": true}}, "at0011": {"value": {"text": "Something, somtehing"}}}}, "at0004": {"items": {"at0005": {"value": {"text": "Bla Bla Bla"}}, "at0006": {"value": {"selected": true}}}}, "at0007": {"items": {"at0009": {"value": {"selected": false}}, "at0008": {"value": {"text": "Yada yada"}}}}}}]}}}]}]}}, "archetype_class": "openEHR-EHR-OBSERVATION.fast_scan.v1"}}}, "archetype_class": "openEHR-EHR-SECTION.report_demo_most.v1"}, "creation_time": 1493717528.054957, "patient_id": "plqa4mckhd33nslx4vplm5hklwe36fqi", "record_id": "f65a2f54f5404ef8808f8eb5acc61465", "version": 1, "active": true, "last_update": 1493717528.054957}, "SUCCESS": true}


In [7]:
#Check new EHR entry for current medicalrecord patient
payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
    params=payload
)
try:
    data = json.loads(r.text)
except ValueError:
    print r.text
else:
    print 'number of records: %s ' % len(data['RECORD'])
    pprint.pprint(data)
    record_id = data['RECORD']['ehr_records'][-1]['record_id']
    #print 'id of last record: %s' % record_id
    #print data['RECORD']['ehr_records'][-1]

number of records: 5 
{u'RECORD': {u'active': True,
             u'creation_time': 1493717432.802065,
             u'ehr_records': [{u'active': True,
                               u'creation_time': 1493717528.054957,
                               u'ehr_data': {u'archetype_class': u'openEHR-EHR-SECTION.report_demo_most.v1',
                                             u'archetype_details': {}},
                               u'last_update': 1493717528.054957,
                               u'patient_id': u'plqa4mckhd33nslx4vplm5hklwe36fqi',
                               u'record_id': u'f65a2f54f5404ef8808f8eb5acc61465',
                               u'version': 1}],
             u'last_update': 1493717528.268835,
             u'record_id': u'plqa4mckhd33nslx4vplm5hklwe36fqi'},
 u'SUCCESS': True}


In [34]:
#Get new EHR record
payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/{record_id}/'.
    format(server_port=server_port, patient_id=patient_id, record_id=record_id), 
    params=payload
)
print r.text
data = json.loads(r.text)
pprint.pprint(data)

{"RECORD": {"ehr_data": {"archetype_details": {"items": {"at0001": {"archetype_details": {"data": {"at0001": [{"events": [{"at0002": {"data": {"at0003": [{"items": {"at0013": {"items": {"at0014": {"value": {"text": "Ill Illr"}}, "at0015": {"value": {"selected": false}}}}, "at0010": {"items": {"at0012": {"value": {"selected": true}}, "at0011": {"value": {"text": "Something, somtehing"}}}}, "at0004": {"items": {"at0005": {"value": {"text": "Bla Bla Bla"}}, "at0006": {"value": {"selected": true}}}}, "at0007": {"items": {"at0009": {"value": {"selected": false}}, "at0008": {"value": {"text": "Yada yada"}}}}}}]}}}]}]}}, "archetype_class": "openEHR-EHR-OBSERVATION.fast_scan.v1"}}}, "archetype_class": "openEHR-EHR-SECTION.report_demo_most.v1"}, "creation_time": 1490726979.80775, "patient_id": "qxp5owqkf55gfncdp43mmq3h4twm3pjg", "record_id": "b44552a86642473a9db81bb208713eea", "version": 1, "active": true, "last_update": 1490726979.80775}, "SUCCESS": true}
{u'RECORD': {u'active': True,
        

In [35]:
#Create and delete new ehr medical record
params = {'access_token': access_token}

ehr_data = {
  "archetype_class": "openEHR.TEST-EVALUATION.v1",
  "archetype_details": {
    "at0001": "val1",
    "at0002": "val2"
  }
}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/'.
    format(server_port=server_port, patient_id=patient_id), 
    params=params, json=ehr_data
)
data = json.loads(r.text)
record_id = data['RECORD']['record_id']

payload = {'access_token': access_token}
r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/records/{record_id}/'.
        format(server_port=server_port, patient_id=patient_id, record_id=record_id), 
    params=payload, data={})

print r.text

{"MESSAGE": "EHR record with ID 70d81fbb4212448a93fe882925679313 successfully hidden", "SUCCESS": true}


In [36]:
#Create and delete patient, and ehr patient

#Create medical record patient
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text
data = json.loads(r.text)
patient_id = data['patient']['uuid']

#Create ehr patient record 
payload = {'access_token': access_token}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
params=payload)
print r.text

#Delete ehr patient record

r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
params=payload, data={})

print r.text

{"patient": {"demographic_uuid": "ltlr4ca7xo7x32gmfbwdkc7is4ronr6f", "ehr_uuid": "uqzotqswcpjtvqht4vla4r4kubxkqy7v", "uuid": "iygyjyvyx7pmi7xnx6kdmznkr4t64yxb"}, "success": true}
{"RECORD": {"record_id": "uqzotqswcpjtvqht4vla4r4kubxkqy7v", "active": true, "ehr_records": [], "creation_time": 1490727911.642054, "last_update": 1490727911.642054}, "SUCCESS": true}
{"RECORD": {"record_id": "uqzotqswcpjtvqht4vla4r4kubxkqy7v", "active": false, "ehr_records": [], "creation_time": 1490727911.642054, "last_update": 1490727911.838461}, "SUCCESS": true}


In [37]:
r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/ehr/{patient_id}/'.
    format(server_port=server_port, patient_id=patient_id), 
params=payload, data={})

print r.text

{"RECORD": {"record_id": "uqzotqswcpjtvqht4vla4r4kubxkqy7v", "active": false, "ehr_records": [], "creation_time": 1490727911.642054, "last_update": 1490727911.838461}, "SUCCESS": true}
